# 라이브러리 로드

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

import warnings
from pathlib import Path

from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
pd.set_option('max_columns', 100)
pd.set_option('display.precision', 4)

warnings.filterwarnings('ignore')

# 학습데이터 로드

In [4]:
data_dir = Path('../data/dacon-novel-author-classification')
feature_dir = Path('../build/feature')
val_dir = Path('../build/val')
tst_dir = Path('../build/tst')
sub_dir = Path('../build/sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5
seed = 2020

In [5]:
algorithm_name = 'ada'
feature_name = 'stacking-layer1-stopwords-no-ml'
model_name = f'{algorithm_name}_{feature_name}'

feature_target_file = feature_dir / f'feature_target.csv'

stacking_oof_pred_file = val_dir / f'{model_name}_oof_pred.csv'
stacking_test_pred_file = tst_dir / f'{model_name}_test_pred.csv'
stacking_submission_file = sub_dir / f'{model_name}_submission.csv'

# Stacking feature 생성

In [6]:
def load_feature(model_names, number_of_ver=None, kind=None):
    oof_list = []
    test_list = []
    
    if number_of_ver==None or kind==None:
        print('error')
        return None
    
    # 딥러닝 시리즈 4가지 버전
    if kind == 0:
        for model in model_names:
            print(f'load {model}_cv')
            for i in range(1,number_of_ver+1):
                oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv', delimiter=','))
                test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
    
    # 로지스틱 회귀 6가지 버전
    elif kind == 1:
        for model in model_names:
            print(f'load {model}_cv')
            for i in range(1, number_of_ver+1):
                oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv', delimiter=','))
                test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))

    # 신경망 기반 불용어 처리 21가지 버전 또는 머신러닝 기반 불용어 처리 18가지 버전
    elif kind == 2:
        for model in model_names:
            print(f'load {model}_cv')
            if model.find('feature') != -1:
                for i in range(2,5):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('tfidf') != -1:
                for i in range(1,4):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('hashing') != -1:
                for i in range(1,4):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('bow') != -1:
                for i in range(1,4):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
    
    # 신경만 기반 불용어 처리 X 13가지 버전 또는 머신러닝 기반 불용어 처리 X 18가지 버전
    elif kind == 3:
        for model in model_names:
            print(f'load {model}_cv')
            if model.find('feature') != -1:
                for i in range(1,2):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('tfidf') != -1:
                for i in range(3,6):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('hashing') != -1:
                for i in range(3,6):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('bow') != -1:
                for i in range(3,6):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
    
    
    return oof_list, test_list

In [ ]:
model_names = ['mlp_tfidf', 'mlp_hashing', 'mlp_bow','lr_tfidf','lr_hashing','lr_bow']

# 의미없는 값
number_of_ver = -1
kind = 3

all_oof, all_test = load_feature(model_names, number_of_ver, kind)
all_oof = np.concatenate(all_oof, axis=1)
all_test = np.concatenate(all_test, axis=1)
all_oof.shape, all_test.shape

In [21]:
y = pd.read_csv(feature_target_file, index_col=0, usecols=['index',target_col]).values.flatten()
y.shape

(54879,)

# 스태킹

- 각 oof 마다 fold별로 logloos 변동이 있으므로 최대한 정보를 뽑아내고자 스태킹을 함.

In [22]:
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 1000,
    'max_features' : 'sqrt',
    'verbose' : 0,
    'random_state': 2020
}

rf_clf = RandomForestClassifier(**rf_params)

# Ada Boost Classifier parameters
ada_params = {
    'base_estimator': rf_clf,
    'n_estimators': 1000,
    'learning_rate': 0.1,
    'algorithm': 'SAMME.R',
    'random_state': 2020
}

In [23]:
datasets = [(all_oof, all_test, y)]

mlogloss = []

ada_oof_preds = []
ada_test_preds = []

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

for number, (X, test , y) in enumerate(datasets, 1):
    print(f'start : {number}')
    
    ada_oof_pred = np.zeros((X.shape[0], n_class))
    ada_test_pred = np.zeros((test.shape[0], n_class))
    
    for i, (i_trn, i_val) in enumerate(cv.split(X, y), 1):
        print(f'training model for CV #{i}')
        
        X_train , X_val = X[i_trn], X[i_val]
        y_train, y_val = y[i_trn], y[i_val]
        
        clf = AdaBoostClassifier(**ada_params)
        clf.fit(X_train,y_train)
                
        ada_oof_pred[i_val, :] = clf.predict_proba(X_val)
        ada_test_pred += clf.predict_proba(test) / n_fold
        mlogloss.append(clf.best_score)
    ada_oof_preds.append(ada_oof_pred)
    ada_test_preds.append(ada_test_pred)
    
    print(f'end : {number}')

start : 1
training model for CV #1
[07:50:38] WARNING: ../src/learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-mlogloss:1.43528	val-mlogloss:1.43988
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[82]	train-mlogloss:0.28940	val-mlogloss:0.42821

training model for CV #2
[07:50:45] WARNING: ../src/learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]

In [24]:
for i,j in enumerate(ada_oof_preds,1):
    print(f'logloss = {log_loss(pd.get_dummies(y),j):8.4f}')
    print(f'accuracy = {accuracy_score(y, np.argmax(j,axis=1))*100:8.4f}')
print('mean logloss = ',np.mean(mlogloss))

logloss =   0.4271
accuracy =  84.6590
mean logloss =  0.4252668


# 제출 파일 및 기타 파일 생성

In [26]:
# submission 파일 생성

sub = pd.read_csv(sample_file,index_col=0)
sub[sub.columns] = ada_test_preds[0]
sub.to_csv(stacking_submission_file)

In [27]:
# stacking_oof_pred 파일 생성

np.savetxt(stacking_oof_pred_file, ada_oof_preds[0],fmt='%.18f', delimiter=',')

In [28]:
# stacking_test_pred 파일 생성

np.savetxt(stacking_test_pred_file, ada_test_preds[0],fmt='%.18f', delimiter=',')